In [1]:
from PIL import Image
import cv2
import glob
import os

In [2]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, transforms
from torch.optim import Adam
from torch.autograd import Variable



In [ ]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/proccessed_photo.zip", 'r') as zip_ref:
  zip_ref.extractall("/content/drive/MyDrive/proccessed_photo")



In [4]:
random_state = 19
img_size = 256
batch_size = 64
data_dir = "proccessed_photo"

transform = transforms.Compose([
    transforms.Resize(size=(img_size,img_size)),
    transforms.ToTensor()
])

data = datasets.ImageFolder(root=data_dir,
                            transform=transform)

data_train, data_test = random_split(data, [0.7,0.3])
train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(data_val, batch_size=8, shuffle=True)
test_loader = DataLoader(data_test, batch_size=batch_size, shuffle=True)
print(len(train_loader))


165


In [22]:
import torchvision.models as models

weights = models.ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights = weights).to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [23]:
for param in model.parameters():
  param.requires_grad = False

unfrozen_layers = ['fc']
for name, param in model.named_parameters():
  if any(layer_name in name for layer_name in unfrozen_layers):
    param.requires_grad = True

In [24]:
num_classes = 15
linear = torch.nn.Linear(1000, 512)
linear2 = torch.nn.Linear(512, num_classes)
model = torch.nn.Sequential(model, linear, linear2)
for name, m in model.named_modules():
    print(name)



0
0.conv1
0.bn1
0.relu
0.maxpool
0.layer1
0.layer1.0
0.layer1.0.conv1
0.layer1.0.bn1
0.layer1.0.conv2
0.layer1.0.bn2
0.layer1.0.conv3
0.layer1.0.bn3
0.layer1.0.relu
0.layer1.0.downsample
0.layer1.0.downsample.0
0.layer1.0.downsample.1
0.layer1.1
0.layer1.1.conv1
0.layer1.1.bn1
0.layer1.1.conv2
0.layer1.1.bn2
0.layer1.1.conv3
0.layer1.1.bn3
0.layer1.1.relu
0.layer1.2
0.layer1.2.conv1
0.layer1.2.bn1
0.layer1.2.conv2
0.layer1.2.bn2
0.layer1.2.conv3
0.layer1.2.bn3
0.layer1.2.relu
0.layer2
0.layer2.0
0.layer2.0.conv1
0.layer2.0.bn1
0.layer2.0.conv2
0.layer2.0.bn2
0.layer2.0.conv3
0.layer2.0.bn3
0.layer2.0.relu
0.layer2.0.downsample
0.layer2.0.downsample.0
0.layer2.0.downsample.1
0.layer2.1
0.layer2.1.conv1
0.layer2.1.bn1
0.layer2.1.conv2
0.layer2.1.bn2
0.layer2.1.conv3
0.layer2.1.bn3
0.layer2.1.relu
0.layer2.2
0.layer2.2.conv1
0.layer2.2.bn1
0.layer2.2.conv2
0.layer2.2.bn2
0.layer2.2.conv3
0.layer2.2.bn3
0.layer2.2.relu
0.layer2.3
0.layer2.3.conv1
0.layer2.3.bn1
0.layer2.3.conv2
0.layer2.3

In [25]:
from torchinfo import summary

summary(model, (1, 3, 256, 256))


Layer (type:depth-idx)                        Output Shape              Param #
Sequential                                    [1, 15]                   --
├─ResNet: 1-1                                 [1, 1000]                 --
│    └─Conv2d: 2-1                            [1, 64, 128, 128]         (9,408)
│    └─BatchNorm2d: 2-2                       [1, 64, 128, 128]         (128)
│    └─ReLU: 2-3                              [1, 64, 128, 128]         --
│    └─MaxPool2d: 2-4                         [1, 64, 64, 64]           --
│    └─Sequential: 2-5                        [1, 256, 64, 64]          --
│    │    └─Bottleneck: 3-1                   [1, 256, 64, 64]          (75,008)
│    │    └─Bottleneck: 3-2                   [1, 256, 64, 64]          (70,400)
│    │    └─Bottleneck: 3-3                   [1, 256, 64, 64]          (70,400)
│    └─Sequential: 2-6                        [1, 512, 32, 32]          --
│    │    └─Bottleneck: 3-4                   [1, 512, 32, 32]       

In [27]:
model = model.to(device)
model = nn.DataParallel(model)
optimizer = Adam(model.parameters(), lr = 0.001, weight_decay = 0.0001)
loss_func = nn.CrossEntropyLoss()

epochs = 50

In [29]:
best_accuracy =0.0
for epoch in range(epochs):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)

        train_accuracy += int(torch.sum(prediction == labels.data))
        if i % 5 == 0:
            print('Train Epoch {}: [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, i * len(images), len(train_loader.dataset),
                100. * i / len(train_loader), loss.item()))

    train_accuracy = train_accuracy / (len(train_loader) * batch_size)
    train_loss = train_loss / (len(train_loader) * batch_size)


    model.eval()
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=images.to(device)
            labels=labels.to(device)

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/(len(test_loader) * batch_size)


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        name = 'best_checkpoint' + str(test_accuracy) + '.model'
        torch.save(model.state_dict(),'best_checkpoint.model', name)
        best_accuracy=test_accuracy

Train Epoch 0: [0/10517 (0%)]	Train Loss: 2.937553
Train Epoch 0: [320/10517 (3%)]	Train Loss: 2.700081
Train Epoch 0: [640/10517 (6%)]	Train Loss: 2.383052
Train Epoch 0: [960/10517 (9%)]	Train Loss: 2.036989
Train Epoch 0: [1280/10517 (12%)]	Train Loss: 2.461080
Train Epoch 0: [1600/10517 (15%)]	Train Loss: 2.639570
Train Epoch 0: [1920/10517 (18%)]	Train Loss: 2.444952
Train Epoch 0: [2240/10517 (21%)]	Train Loss: 2.028296
Train Epoch 0: [2560/10517 (24%)]	Train Loss: 2.330147
Train Epoch 0: [2880/10517 (27%)]	Train Loss: 2.226793
Train Epoch 0: [3200/10517 (30%)]	Train Loss: 1.838312
Train Epoch 0: [3520/10517 (33%)]	Train Loss: 2.481421
Train Epoch 0: [3840/10517 (36%)]	Train Loss: 2.482744
Train Epoch 0: [4160/10517 (39%)]	Train Loss: 2.006682
Train Epoch 0: [4480/10517 (42%)]	Train Loss: 2.358213
Train Epoch 0: [4800/10517 (45%)]	Train Loss: 2.789288
Train Epoch 0: [5120/10517 (48%)]	Train Loss: 2.053547
Train Epoch 0: [5440/10517 (52%)]	Train Loss: 2.064105
Train Epoch 0: [5760

In [ ]:
model.load_state_dict(torch.load('best_checkpoint.model'))
model.eval()

Simple_net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=524288, out_features=3, bias=True)
)